Чтобы увеличить скорость вычислений, переключите Accelerator в ноутбуке на kaggle на один из GPU, после чего запустите выполнение кода.

In [1]:
import glob

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms

In [2]:
# base_dir = '/kaggle/input/cian-datafest-2019/train.zip'

# import zipfile
# with zipfile.ZipFile(base_dir, 'r') as z:
#     z.extractall()

In [3]:
if torch.cuda.is_available():
    dev = 'cuda:0'
else:
    dev = 'cpu'
device = torch.device(dev)

В PyTorch удобно проводить обработку и разбиение данных в помощью функции `torch.utils.data.DataLoader`. Чтобы ей воспользоваться, нужно создать класс с методами `__len__` и `__getitem__`, которые возвращают длину всей выборки и один объект выборки соответственно.

Создадим класс для чтения данных ЦИАН с kaggle. Преобразования картинок можно производить через модуль `torchvision.transforms`.

In [4]:
class CIANDataset(torch.utils.data.Dataset):   
    def __init__(self, filenames):
        
        self.transform = transforms.Compose(
            [
                transforms.Resize((10,10)), 
                transforms.ToTensor()
            ]
        )
        
        self.filenames = filenames
        self.y = np.array(
            ['indoor' in filename for filename in self.filenames]
        ).astype('float32')

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        
        img_name = self.filenames[index]
        img = Image.open(img_name).convert('RGB')
        img = self.transform(img).reshape(-1) / 255.
        
        target = self.y[index]
        
        return img, target

Разобьём данные на обучение и контроль. В переменной `filenames` хранятся названия всех файлов директорий indoor и outdoor. Функция `torch.utils.data.Subset` извлекает каждое 6-е название файла. Всего получится 9221 название. Из них 6000 случайных изображений возьмём для обучения с помощью функции `torch.utils.data.random_split`, оставшиеся 3221 будем использовать для оценки качества модели.

In [5]:
filenames = glob.glob('./train/*/*')

trainset = torch.utils.data.Subset(filenames, list(range(0, len(filenames), 6)))

train_set, val_set = torch.utils.data.random_split(trainset, [6000, 3221])

Через `DataLoader` будем разбивать обучающую и контрольную выборки на батчи по 64 изображения.

In [6]:
train_dataset = CIANDataset(train_set)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CIANDataset(val_set)
val_len = val_dataset.__len__()
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_len, shuffle=False)

Далее реализуем архитектуру полносвязной нейронной сети и обучим её, пользуясь готовым разбиением на батчи `train_loader`.

<div class="alert alert-info">

<h3> Задание</h3>
<p></p>
Реализуйте свою полносвязную нейронную сеть. Попробуйте добиться наилучшего качества предсказаний на контрольной выборке (меняйте количество слоёв, количество нейронов на скрытых слоях, функции активации между слоями, шаг обучения, количество эпох). На каждой эпохе выводите текущие значения функции потерь на обучающей и контрольной выборках, чтобы не допустить переобучения.
 <p></p>
</div>

In [7]:
class FCNN(nn.Module):
    def __init__(self, input_size=300, num_classes=1):
        super(FCNN, self).__init__()
        
        self.layer_1 = torch.nn.Linear(input_size, 100)
        self.layer_2 = torch.nn.Linear(100, 100)
        self.layer_out = torch.nn.Linear(100, 1)
        
        self.activation = torch.nn.ReLU()
        
    def forward(self, inputs):
        output_1 = self.activation(self.layer_1(inputs))
        output_2 = self.activation(self.layer_2(output_1))
        output = self.layer_out(output_2)
        
        return output

In [8]:
np.random.seed(42)
torch.manual_seed(42)

model = FCNN().to(device)

# criterion, optimizer (Adam)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 20

model.train()
for epoch in range(1, num_epochs + 1):
    for num_batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.reshape(-1, 1).to(device)
        
        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, labels)
        print('epoch: ', epoch + 1, ' num_batch: ', num_batch, ' loss: ', loss.item())
        loss.backward()
        optimizer.step()

epoch:  2  num_batch:  0  loss:  0.6951886415481567
epoch:  2  num_batch:  1  loss:  0.6952397227287292
epoch:  2  num_batch:  2  loss:  0.6930358409881592
epoch:  2  num_batch:  3  loss:  0.6930614709854126
epoch:  2  num_batch:  4  loss:  0.692608654499054
epoch:  2  num_batch:  5  loss:  0.6914587020874023
epoch:  2  num_batch:  6  loss:  0.686430037021637
epoch:  2  num_batch:  7  loss:  0.7036788463592529
epoch:  2  num_batch:  8  loss:  0.7084642648696899
epoch:  2  num_batch:  9  loss:  0.6813260316848755
epoch:  2  num_batch:  10  loss:  0.694963276386261
epoch:  2  num_batch:  11  loss:  0.7004272937774658
epoch:  2  num_batch:  12  loss:  0.7013665437698364
epoch:  2  num_batch:  13  loss:  0.689704179763794
epoch:  2  num_batch:  14  loss:  0.6887322068214417
epoch:  2  num_batch:  15  loss:  0.6947858333587646
epoch:  2  num_batch:  16  loss:  0.6850475072860718
epoch:  2  num_batch:  17  loss:  0.6870514154434204
epoch:  2  num_batch:  18  loss:  0.6885853409767151
epoch: 

In [9]:
# оценка качества
total = val_len
activation = torch.nn.ReLU()
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.reshape(-1, 1).to(device)
        pred = activation(model(images))
        correct = torch.sum(torch.eq((pred > 0).int(), labels).int())

    print('Accuracy: {:.4f} %'.format(100 * correct / total))

Accuracy: 86.8364 %
